In [2]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [91]:

chats = [
    (1, 1, 'damu', 'Qué es esto?'),
    (2, 2, 'martin', 'Un chat!'),
    (3, 1, 'damu', 'Ahhh! Y de donde salio? Whatsapp?'),
    (4, 2, 'martin', 'Sí! Cómo sabias?'),
    (5, 1, 'damu', 'Adivine'),
    (6, 3, 'luis', 'Hola!')
]

In [92]:
data = sc.parallelize(chats)

In [93]:
data.collect()

[(1, 1, 'damu', 'Qué es esto?'),
 (2, 2, 'martin', 'Un chat!'),
 (3, 1, 'damu', 'Ahhh! Y de donde salio? Whatsapp?'),
 (4, 2, 'martin', 'Sí! Cómo sabias?'),
 (5, 1, 'damu', 'Adivine'),
 (6, 3, 'luis', 'Hola!')]

In [9]:
data.count()

6

In [12]:
data.take(3)

[(1, 1, 'damu', 'Qué es esto?'),
 (2, 2, 'martin', 'Un chat!'),
 (3, 1, 'damu', 'Ahhh! Y de donde salio? Whatsapp?')]

In [27]:
def func(x):
    #print(x)
    #print(' x[0]: ', x[0])
    #print(' x[1]: \n ', x[1])
    return x[0]

data.takeOrdered(10,key = func)

[(1, 1, 'damu', 'Qué es esto?'),
 (2, 2, 'martin', 'Un chat!'),
 (3, 1, 'damu', 'Ahhh! Y de donde salio? Whatsapp?'),
 (4, 2, 'martin', 'Sí! Cómo sabias?'),
 (5, 1, 'damu', 'Adivine'),
 (6, 3, 'luis', 'Hola!')]

In [29]:
data.first()

(1, 1, 'damu', 'Qué es esto?')

In [41]:
data.countByKey()

defaultdict(int, {1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 7: 1})

In [42]:
data.countByValue()

defaultdict(int,
            {(1, 1, 'damu', 'Qué es esto?'): 1,
             (2, 2, 'martin', 'Un chat!'): 1,
             (3, 1, 'damu', 'Ahhh! Y de donde salio? Whatsapp?'): 1,
             (4, 2, 'martin', 'Sí! Cómo sabias?'): 1,
             (5, 1, 'damu', 'Adivine'): 1,
             (7, 3, 'luis', 'Hola!'): 1})

In [52]:
data.reduce(lambda x,y: x+y)

(1,
 1,
 'damu',
 'Qué es esto?',
 2,
 2,
 'martin',
 'Un chat!',
 3,
 1,
 'damu',
 'Ahhh! Y de donde salio? Whatsapp?',
 4,
 2,
 'martin',
 'Sí! Cómo sabias?',
 5,
 1,
 'damu',
 'Adivine',
 7,
 3,
 'luis',
 'Hola!')

In [60]:
# El map, aplica a cada fila del rdd algo, usando ese mismo valor. Permite transformar.
data.map(lambda x: x[0]*x[0]).reduce(lambda x,y: x+y)

104

In [64]:
# basicamente es el que me permite aplicar condiciones
data.filter(lambda x: x[0]>3).collect()
# x es el dato completo
# x[0] es el primero
# x[1] ..

[(4, 2, 'martin', 'Sí! Cómo sabias?'),
 (5, 1, 'damu', 'Adivine'),
 (7, 3, 'luis', 'Hola!')]

In [69]:
data.map(lambda x: [a for a in range(0,x[0])]).collect()

[[0], [0, 1], [0, 1, 2], [0, 1, 2, 3], [0, 1, 2, 3, 4], [0, 1, 2, 3, 4, 5, 6]]

In [77]:
# data.flatmap(func) esto serviria si lo de adentro devuelve una lista o algo asi

In [79]:
data.distinct().collect()

[(3, 1, 'damu', 'Ahhh! Y de donde salio? Whatsapp?'),
 (1, 1, 'damu', 'Qué es esto?'),
 (2, 2, 'martin', 'Un chat!'),
 (5, 1, 'damu', 'Adivine'),
 (7, 3, 'luis', 'Hola!'),
 (4, 2, 'martin', 'Sí! Cómo sabias?')]

In [84]:
# data.reduceByKey(lambda x: x[0]+y[0]).collect()

In [87]:
# data.groupByKey()

# 2016-01 Recuperatorio
Una red social almacena el contenido de los chats entre sus usuarios en un RDD que tiene registros con el siguiente formato: (chat_id, user_id, nickname, text). Queremos saber cuál es el usuario (user_id) que mas preguntas hace en sus chats, contabilizamos una pregunta por cada caracter “?” que aparezca en el campo text. Programar en Spark un programa que identifique al usuario preguntón.

In [99]:
# Con el map, lo llevo a algo de la forma (k,v) donde k es el id de usuario y v es la cantidad de veces de ?
d = data.map(lambda x: (x[1], x[3].count('?')))\
    #.reduceByKey(lambda x, y: x+y)\
    #.reduce(lambda x, y: x if (x[1] > y[1]) else y)

[(1, 1), (2, 0), (1, 2), (2, 1), (1, 0), (3, 0)]

In [101]:
# Luego, reduzco por key, junto todas key sumandole sus counts. Deberia quedarme [(1,3),(2,1),(3,0)]
d = d.reduceByKey(lambda x,y: x+y)

[(1, 3), (2, 1), (3, 0)]

In [102]:
# Me quedo con el mas grande en count. Nose si esto es lo mas efectivo, o deberia hacer un takeordered quizas.
pregunton = d.reduce(lambda x,y: x if x[1]>y[1] else y)

(1, 3)

# 2016-01 Parcial
UBER almacena en un cluster todos los datos sobre el movimiento y viajes de todos sus vehículos. Existe un proceso que nos devuelve un RDD llamado trip_summary con los siguientes campos: (driver_id, car_id, trip_id, customer_id, date (YYYYMMDD), distance_traveled), Programar usando Py Spark un programa que nos indique cual fue el conductor con mayor promedio de distancia recorrida por viaje para Abril de 2016.



In [109]:
trips = [
    (1, 1, 1, 1, '20160101', 10),
    (2, 2, 2, 2, '20160202', 20),
    (1, 1, 3, 1, '20160402', 15),
    (1, 1, 4, 3, '20160405', 20),
    (2, 2, 5, 4, '20160410', 25),
    (3, 3, 6, 3, '20160415', 15),
    (2, 2, 7, 1, '20160420', 40),
    (3, 3, 8, 2, '20160505', 80)
]

In [110]:
data = sc.parallelize(trips)

In [122]:
# con el filter me quedo solo los de abril
data.filter(lambda x: x[4]>='20160401' and x[4]<'20160501')\
    .map(lambda x: (x[0], (1, x[5]) ))\
    .reduceByKey(lambda x,y: (x[0]+y[0], x[1] + y[1]) )\
    .reduce(lambda x,y: x if x[1][1]/x[1][0] > y[1][1]/y[1][0] else y   )

(2, (2, 65))

# 2015-02 2do Recuperatorio
Un telescopio registra automaticamente la luminosidad de distintas estrellas generando un RDD con registros de tipo (star_id, magnitude,spectral_type, timestamp). Queremos obtener un listado de estrellas que tienen el mismo tipo espectral e igual promedio de magnitud una vez redondeado el mismo a un decimal. El resultado debe ser una lista en donde cada elemento de la lista es una lista de ids de estrellas similares.

In [123]:
stars = [
    (1, 5, 1, 1),
    (2, 10, 1, 1),
    (3, 6, 1, 1),
    (4, 5.5, 2, 1),
    (1, 6, 1, 2),
    (2, 9, 1, 2),
    (3, 5, 1, 2),
    (1, 5.5, 1, 3),
    (2, 11, 1, 3),
    (3, 5.5, 1, 3)
]

In [124]:
data = sc.parallelize(stars)

In [152]:
data.map(lambda x: (x[0],(x[1],x[2], 1)))\
    .reduceByKey(lambda x,y: (x[0]+y[0], x[1], x[2]+y[2]))\
    .map(lambda x: ((x[1][1], x[1][0]/x[1][2]) ,x[0]))\
    .groupByKey()\
    .map(lambda x: (x[0], list(x[1])))\
    .collect()
    #.reduceByKey(lambda x,y: )

[((1, 5.5), [1, 3]), ((1, 10.0), [2]), ((2, 5.5), [4])]

In [131]:
data.map(lambda x: (x[0], (x[2], x[1], 1)))\
    .reduceByKey(lambda x, y: (x[0], x[1]+y[1], x[2]+y[2]))\
    .map(lambda x: ((x[1][0], x[1][1]/x[1][2]), x[0]))\
    .groupByKey()\
    .map(lambda x: (x[0], list(x[1])))\
    .collect()

[((1, 5.5), [1, 3]), ((1, 10.0), [2]), ((2, 5.5), [4])]

# 2016-02 Parcial
En este ejercicio queremos programar un sistema que recomiende textos a usuarios en base a sus gustos sobre ciertos términos (palabras).

Se cuenta con un RDD de textos de la forma (docId, texto) donde texto es un string de longitud variable.

Además contamos con un RDD que indica qué términos le gustan o no a cada usuario de la forma (userId, término, score) por ejemplo (23, “calesita”, -2).

Se pide programar en Spark un programa que calcule el score total de cada documento para cada usuario generando un RDD de la forma (userId, docId, score) en donde el score es simplemente la suma de los scores del usuario para los términos que aparecen en el documento.

Puede haber términos en los documentos para los cuales no exista score de algunos usuarios, en estos casos simplemente los consideramos neutros (score=0)

In [153]:
documents = [
    (1, 'pablo honey'),
    (2, 'the bends'),
    (3, 'ok computer'),
    (4, 'kid a'),
    (5, 'amnesiac'),
    (6, 'hail to the thief'),
    (7, 'in rainbows'),
    (8, 'the king of limbs'),
    (9, 'a moon shaped pool')
]

scores = [
    ('thom', 'pablo', 1),
    ('thom', 'honey', 1),
    ('martin', 'pablo', -1),
    ('martin', 'honey', -1),
    ('martin', 'ok', 30),
    ('martin', 'computer', 30)
]

In [160]:

documentsRDD = sc.parallelize(documents)
scoresRDD = sc.parallelize(scores)

In [161]:
words = documentsRDD.flatMap( lambda a: [(word, a[0]) for word in a[1].split()] )
words.collect()

[('pablo', 1),
 ('honey', 1),
 ('the', 2),
 ('bends', 2),
 ('ok', 3),
 ('computer', 3),
 ('kid', 4),
 ('a', 4),
 ('amnesiac', 5),
 ('hail', 6),
 ('to', 6),
 ('the', 6),
 ('thief', 6),
 ('in', 7),
 ('rainbows', 7),
 ('the', 8),
 ('king', 8),
 ('of', 8),
 ('limbs', 8),
 ('a', 9),
 ('moon', 9),
 ('shaped', 9),
 ('pool', 9)]

In [163]:
scores_for_join = scoresRDD.map(lambda x: (x[1], (x[0],x[2])) )
scores_for_join.collect()

[('pablo', ('thom', 1)),
 ('honey', ('thom', 1)),
 ('pablo', ('martin', -1)),
 ('honey', ('martin', -1)),
 ('ok', ('martin', 30)),
 ('computer', ('martin', 30))]

In [164]:
join = words.join(scores_for_join)
join.collect()

[('honey', (1, ('thom', 1))),
 ('honey', (1, ('martin', -1))),
 ('ok', (3, ('martin', 30))),
 ('pablo', (1, ('thom', 1))),
 ('pablo', (1, ('martin', -1))),
 ('computer', (3, ('martin', 30)))]

In [167]:
a = join.map(lambda x: ((x[1][1][0], x[1][0]) , x[1][1][1]) )
a.collect()

[(('thom', 1), 1),
 (('martin', 1), -1),
 (('martin', 3), 30),
 (('thom', 1), 1),
 (('martin', 1), -1),
 (('martin', 3), 30)]

In [168]:
data = a.reduceByKey(lambda x,y: x+y)\
        .map(lambda x: (x[0][0],x[0][1],x[1]))\
        .collect()

In [169]:
data

[('thom', 1, 2), ('martin', 1, -2), ('martin', 3, 60)]

In [170]:
# Si quisiera leer de un archivo: sc.textFile("README.md") esto devuelve el rdd.